## Code for making the cell volume from cells detected and registered to Allen Atlas

In [1]:
import numpy as np
import tifffile as tif
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import ball
%matplotlib inline

In [3]:
#first, make a map of cells
pth = "/jukebox/wang/pisano/tracing_output/antero_4x_analysis/201903_antero_pooled_cell_counts_thalamus/transformed_points/20161205_tp_bl6_lob45_1000r_01/transformed_points/posttransformed_zyx_voxels.npy"
converted_points = np.load(pth)

zyx = np.asarray([(int(xx[0]), int(xx[1]), int(xx[2])) for xx in converted_points]) #cells are counted in horizontal volumes

In [5]:
#read registered volume
vol = '/jukebox/wang/pisano/tracing_output/antero_4x/20161205_tp_bl6_lob45_1000r_01/elastix/20161205_tp_bl6_lob45_1000r_01_4x_647_008na_1hfds_z7d5um_200msec_10povlp_resized_ch00/result.tif'
# vol = "/jukebox/wang/zahra/h129_contra_vs_ipsi/reg_to_allen/20180410_jg52_bl6_lob7_05/cell_to_reg/result.tif"
img = tif.imread(vol)  

In [10]:
#init empty vol 
cell_map = np.zeros_like(img).astype('uint16')

In [11]:
#fill map
for z,y,x in zyx:
    try:
        cell_map[z,y,x] = 1 # no dilation
    except Exception as e:
        print(e)

In [8]:
cell_map.shape # Z Y X 

(540, 640, 352)

In [9]:
cell_map[0].shape

(640, 352)

In [13]:
tif.imsave("/jukebox/wang/ahoag/proposals/NSF/20161205_tp_bl6_lob45_1000r_01_registered_cellmap_single_pixel.tif", cell_map)

In [15]:
cell_map.dtype

dtype('uint16')

In [14]:
# Now create the cloudvolume for this tif file
import os
import numpy as np

from concurrent.futures import ProcessPoolExecutor

from taskqueue import LocalTaskQueue
import igneous.task_creation as tc


from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
from PIL import Image

In [16]:
home_dir = '/jukebox/wang/ahoag/proposals/NSF/neuroglancer'
progress_dir = mkdir(home_dir + '/label_progress_regcells/')
def make_info_file():
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'segmentation', # 'image' or 'segmentation'
        data_type = 'uint16', # can pick any popular uint
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = [ 20000, 20000, 20000 ], # X,Y,Z values in nanometers, 40 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels
        volume_size = [352,640,540], # X,Y,Z size in voxels
    )

    # If you're using amazon or the local file system, you can replace 'gs' with 's3' or 'file'
    vol = CloudVolume('file:///jukebox/wang/ahoag/proposals/NSF/neuroglancer/bucket/proposal/registered_cells', info=info)
    vol.provenance.description = "Show cells registered to the Princeton Mouse atlas"
    vol.provenance.owners = ['ahoag@princeton.edu'] # list of contact email addresses

    vol.commit_info() # generates gs://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates gs://bucket/dataset/layer/provenance json file
    return vol

In [17]:
def process_slice(z):
    print('Processing slice z=',z)
    
    array = cell_map[z].reshape((1,y_dim,x_dim)).T

    cloud_vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))

In [18]:
cloud_vol = make_info_file()
""" tifffile is already loaded """
# image = np.array(tifffile.imread(brainvolume_file),dtype=np.uint16, order='F') # F stands for fortran order
z_dim,y_dim,x_dim = cell_map.shape

done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(cloud_vol.bounds.minpt.z, cloud_vol.bounds.maxpt.z)) # 1 indexed 

to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
# print("Remaining slices to upload are:",to_upload)

with ProcessPoolExecutor(max_workers=8) as executor:
    executor.map(process_slice, to_upload)

Processing slice z= 1
Processing slice z= 0
Processing slice z= 3
Processing slice z= 6
Processing slice z= 2
Processing slice z= 7
Processing slice z= 4
Processing slice z= 5


Uploading: 100%|██████████| 1/1 [00:00<00:00, 36.33it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 46.24it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00, 39.47it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 63.77it/s]





Processing slice z= 8
Processing slice z= 9
Processing slice z= 10


Processing slice z= 11


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading: 100%|██████████| 1/1 [00:00<00:00, 79.23it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Processing slice z= 538


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Processing slice z= 539



Uploading: 100%|██████████| 1/1 [00:00<00:00, 58.18it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 100.89it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00, 99.85it/s]




In [19]:
# make the igenous mesh to display in 3D
def make_mesh():
    # Mesh on 8 cores, use True to use all cores
    cloudpath = cloud_vol.cloudpath
    with LocalTaskQueue(parallel=8) as tq:
      tasks = tc.create_meshing_tasks(cloudpath, mip=0, shape=(256, 256, 256))
      tq.insert_all(tasks)
      tasks = tc.create_mesh_manifest_tasks(cloudpath)
      tq.insert_all(tasks)
    print("Done!")

In [20]:
make_mesh()

Downloading:  93%|█████████▎| 239/257 [00:00<00:00, 277.15it/s]
Downloading: 470it [00:01, 333.90it/s]                         
Downloading: 476it [00:01, 340.40it/s]                         
Downloading: 478it [00:01, 336.03it/s]                         
Downloading: 474it [00:01, 336.54it/s]                         
Downloading: 484it [00:01, 335.50it/s]                         
Downloading: 478it [00:01, 340.10it/s]                         
Downloading: 472it [00:01, 329.08it/s]                         



Downloading: 482it [00:01, 338.48it/s]                         





Downloading:  83%|████████▎ | 213/257 [00:00<00:00, 500.02it/s]
Downloading: 446it [00:00, 621.96it/s]                         
Downloading:  30%|██▉       | 76/257 [00:00<00:00, 758.79it/s]

Downloading: 426it [00:00, 624.15it/s]                         
Downloading:   0%|          | 0/257 [00:00<?, ?it/s]

Downloading:  83%|████████▎ | 214/257 [00:00<00:00, 646.35it/s]
Downloading: 490it [00:00, 670.35it/s]    

Done!


In [23]:
# Start a viewer and see what it looks like
import neuroglancer
from collections import OrderedDict
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

In [16]:
# First need to host the cloudvolume -- did that in the terminal at port 1337

In [24]:
viewer = neuroglancer.Viewer()
# This volume handle can be used to notify the viewer that the data has changed.

with viewer.txn() as s:
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1337'
    )

#     s.layers[0]._json_data['segments']=unique_segments
print(viewer)

http://127.0.0.1:35344/v/a75608e74de967b12d03fb3a6d7a416d18428bfe/


In [25]:
with viewer.txn() as s:
    s.layers['atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )

I loaded the allen atlas on there as well and it looks like the atlas is offset in z by a few hundred planes. Let's look at the allen atlas info file to see what is going on

In [30]:
allen_vol = CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/atlas/allenatlas/')

In [31]:
allen_vol.info

{'data_type': 'uint32',
 'mesh': 'mesh_mip_0_err_40',
 'num_channels': 1,
 'scales': [{'chunk_sizes': [[1024, 1024, 1]],
   'encoding': 'raw',
   'key': '40000_40000_40000',
   'resolution': [40000, 40000, 40000],
   'size': [320, 528, 456],
   'voxel_offset': [0, 0, 1]}],
 'type': 'segmentation'}

Yeah, looks like the x and z dimensions are swapped. Just remake the allen atlas with the correct dimensions and it should be fine. 

In [19]:
ss = neuroglancer.ScreenshotSaver(viewer, '/home/ahoag/ngdemo/screenshots/')
ss.capture()  

KeyboardInterrupt: 